In [20]:
import cv2
import os
import pandas as pd
import numpy as np

def augment_brightness(image, factor=0.7):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv[:, :, 2] = np.clip(hsv[:, :, 2] * factor, 0, 255)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

def augment_flip(image):
    return cv2.flip(image, 1)  # 1 for horizontal flip

def augment_blur(image, kernel_size=(5, 5)):
    return cv2.GaussianBlur(image, kernel_size, 0)

def apply_augmentation(image_path, linear_speed_x, angular_speed_z, is_turning, output_folder, df_augmented):
    image = cv2.imread(image_path)

    if (angular_speed_z >= 0.8 or angular_speed_z <= -0.8) and is_turning == "True":

        for i in range(5):
            augmented_image1 = augment_brightness(image)
            save_path1 = os.path.join(output_folder, os.path.basename(image_path).replace('.jpg', f'_brightness_{i}.jpg'))
            cv2.imwrite(save_path1, augmented_image1)

            augmented_image2 = augment_blur(image)
            save_path2 = os.path.join(output_folder, os.path.basename(image_path).replace('.jpg', f'_blur_{i}.jpg'))
            cv2.imwrite(save_path2, augmented_image2)
            
            # Update CSV for augmented images
            df_augmented_entry = pd.DataFrame({'image name': [os.path.basename(save_path1), os.path.basename(save_path2)],
                                               'linear_speed_x': [linear_speed_x, linear_speed_x],
                                               'angular_speed_z': [angular_speed_z, -angular_speed_z]})
            
            # Append the augmented entry to the existing DataFrame
            df_augmented = pd.concat([df_augmented, df_augmented_entry], ignore_index=True)

    if angular_speed_z >= 0.8 or angular_speed_z <= -0.8:

        for i in range(2):
            augmented_image1 = augment_brightness(image)
            save_path1 = os.path.join(output_folder, os.path.basename(image_path).replace('.jpg', f'_brightness_{i}.jpg'))
            cv2.imwrite(save_path1, augmented_image1)

            augmented_image2 = augment_blur(image)
            save_path2 = os.path.join(output_folder, os.path.basename(image_path).replace('.jpg', f'_blur_{i}.jpg'))
            cv2.imwrite(save_path2, augmented_image2)
            
            # Update CSV for augmented images
            df_augmented_entry = pd.DataFrame({'image name': [os.path.basename(save_path1), os.path.basename(save_path2)],
                                               'linear_speed_x': [linear_speed_x, linear_speed_x],
                                               'angular_speed_z': [angular_speed_z, -angular_speed_z]})
            
            # Append the augmented entry to the existing DataFrame
            df_augmented = pd.concat([df_augmented, df_augmented_entry], ignore_index=True)
    
    elif angular_speed_z <= 0.3 and angular_speed_z >= -0.3:
        # Find the index of the entry in the original DataFrame
        index_to_update = df_original.index[df_original['image name'] == image_file].tolist()[0]

        # Update the entry in the original DataFrame with negated values
        df_original.loc[index_to_update, 'angular_speed_z'] = -angular_speed_z

    return df_augmented, df_original

# Read the original CSV file
csv_file_path = r"C:/Users/ishit/Documents/ROSbot_data_collection/datasets/training_data_3/data1.csv"
df_original = pd.read_csv(csv_file_path)

# Specify the output folder for augmented images
output_folder = r"C:/Users/ishit/Documents/ROSbot_data_collection/datasets/augment_YES/"
os.makedirs(output_folder, exist_ok=True)

# Initialize an empty DataFrame for augmented data
columns = ['image name', 'linear_speed_x', 'angular_speed_z', 'is_turning']
df_augmented = pd.DataFrame(columns=columns)

# Iterate through images in the folder
image_folder_path = r"C:/Users/ishit/Documents/ROSbot_data_collection/datasets/training_data_3"
original_samples = len(df_original)

for image_file in os.listdir(image_folder_path):
    if image_file.endswith('.jpg'):
        image_path = os.path.join(image_folder_path, image_file)
        linear_speed_x = df_original.loc[df_original['image name'] == image_file, 'linear_speed_x'].values[0]
        angular_speed_z = df_original.loc[df_original['image name'] == image_file, 'angular_speed_z'].values[0] 
        is_turning = df_original.loc[df_original['image name'] == image_file, 'is_turning'].values[0] 
        df_augmented, df_original = apply_augmentation(image_path, linear_speed_x, angular_speed_z, is_turning, output_folder, df_augmented)

# Save the augmented DataFrame to CSV
df_augmented.to_csv(os.path.join(output_folder, 'data_augmented.csv'), index=False)
df_original.to_csv(image_folder_path, index=False, mode='w')

C:\Users\ishit\AppData\Local\Temp\ipykernel_2660\3261988653.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_augmented = pd.concat([df_augmented, df_augmented_entry], ignore_index=True)


PermissionError: [Errno 13] Permission denied: 'C:/Users/ishit/Documents/ROSbot_data_collection/datasets/training_data_3'